
# Solving the SApHLP problem using simulated annealing

This notebook inclludes the implementation of the Simulated Annealing (SA) algorithm for a hub location problem. The SA algorithm is a probabilistic technique for approximating the global optimum of a given function. Here, we specifically use it to find an optimal set of hubs in a transportation network.

The algorithm is based on principles from statistical mechanics and the process of annealing in metallurgy. It uses a metaphor of heating a material and then slowly cooling it to decrease defects and increase the size of the constituent crystals to find an optimal solution.

I perform a sensitivity analysis on different parameters such as the temperature, cooling rate (alpha), and number of iterations. The datasets used include different sizes of transportation networks (ranging from 10 to 100 nodes) with corresponding flow and cost matrices.

In [32]:
# load the data
%run data_preparation.ipynb

In [ ]:
# import libraries
import time
import csv
import numpy as np

# load the functions
from hub_operations import neighbour, neighbour3, initial_solution, total_cost

In [33]:
# # Define the constants
RESULTS_FILE = 'SA_results.csv'
FM_NAMES = ['CAB_25_fm', 'TR_55_fm', 'TR_81_fm', 'RGP_100_fm']
NUMBER_OF_HUBS_DICT = {'CAB_10_fm': [3], 'CAB_25_fm': [3,5], 'TR_55_fm': [3,5], 'TR_81_fm': [5,7], 'RGP_100_fm': [7,10]}
TEMPERATURES = [1_000_000_000, 10_000_000_000, 100_000_000_000]
ALPHAS = [0.96, 0.97, 0.98, 0.99]
MAX_TS_DICT = {'CAB_10_fm': [500], 'CAB_25_fm': [2000], 'TR_55_fm': [2500], 'TR_81_fm': [5000], 'RGP_100_fm': [6000]}

# # Write headers to results file
# with open(RESULTS_FILE, 'a') as f:
#     writer = csv.writer(f)
#     writer.writerow(['SA Algorithm', 'fm_name', 'cm_name', 'number_of_hubs', 'max_iter', 'T', 'temp', 'alpha', 'best_cost[-1]', 'best_solution[-1]', 'best_cost',  'best_solution', 'total_time'])


In [34]:
# Store the matrices in a dictionary
matrices = {
    'CAB_10_fm': CAB_10_fm,
    'CAB_10_cm': CAB_10_cm,
    'CAB_25_fm': CAB_25_fm,
    'CAB_25_cm': CAB_25_cm,
    'TR_55_fm': TR_55_fm,
    'TR_55_cm': TR_55_cm,
    'TR_81_fm': TR_81_fm,
    'TR_81_cm': TR_81_cm,
    'RGP_100_fm': RGP_100_fm,
    'RGP_100_cm': RGP_100_cm
}

results_file = 'SA_results.csv'

with open(results_file, 'a') as f:
    writer = csv.writer(f)
    writer.writerow(['SA Algorithm', 'fm_name', 'cm_name', 'number_of_hubs', 'max_iter', 'T', 'temp', 'alpha', 'best_cost[-1]', 'best_solution[-1]', 'best_cost',  'best_solution', 'total_time'])

# Rest of the code goes here


In [35]:
# Simulated annealing algorithm
def simulated_annealing(T, alpha, max_iter, fm, fm_name, cm, cm_name, number_of_hubs):
    """Simulated annealing algorithm for hub location problem.

    Args:
    T: Initial temperature for annealing process.
    alpha: Cooling rate.
    max_iter: Maximum number of iterations.
    fm: Flow matrix.
    fm_name: Name of the flow matrix.
    cm: Cost matrix.
    cm_name: Name of the cost matrix.
    number_of_hubs: Number of hubs in the problem.

    Returns:
    best_solution: Best solution found.
    best_cost: Cost of the best solution.
    current_cost: Cost of the current solution.
    """
    # Initialize solution
    initial_sol = initial_solution(fm, cm, number_of_hubs)

    # Start timer
    start_time = time.time()

    # Set initial solution and cost
    current_solution = [initial_sol]
    current_cost = [total_cost(fm , cm, current_solution[-1])]

    # Set initial best solution and cost
    best_solution = [current_solution[-1]]
    best_cost = [current_cost[-1]]

    # Initialize temperature
    temp = [T]

    # Counter for accepted worse solutions
    counter = 0

    # Iteratively generate and evaluate solutions
    for i in range(max_iter):
        # Generate neighbour solution
        new_solution = neighbour(current_solution[-1]) if np.random.random() <= 0.5 else neighbour3(current_solution[-1])

        # Calculate cost of new solution
        new_cost = total_cost(fm, cm, new_solution)

        # If new solution is better, accept it and decrease temperature
        if new_cost < current_cost[-1]:
            current_solution.append(new_solution)
            current_cost.append(new_cost)
            temp.append(alpha * temp[-1])

            # If new solution is the best so far, update best solution and cost
            if new_cost < best_cost[-1]:
                best_solution.append(new_solution)
                best_cost.append(new_cost)
        else:
            # If new solution is worse, accept it with certain probability
            p = np.exp(-(new_cost - current_cost[-1]) / temp[-1])
            if np.random.random() <= p:
                current_solution.append(new_solution)
                current_cost.append(new_cost)
                temp.append(alpha * temp[-1])
                counter += 1

            # Append current best solution and cost
            best_solution.append(best_solution[-1])
            best_cost.append(best_cost[-1])

    # Stop timer and calculate total time
    total_time = time.time() - start_time
    print(f'Timer ended. Total time: {total_time}')
    print(f'Worse solution accepted: {counter} times')
    print(f'Best cost: {best_cost[-1]}')

    # Write results to file
    with open(RESULTS_FILE, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(['SA Algorithm', fm_name, cm_name, number_of_hubs, max_iter, T, temp, alpha, best_cost[-1], best_solution[-1], best_cost, best_solution, total_time])

    return best_solution, best_cost, current_cost




In [36]:
def run_simulated_annealing(iterations, fm_names, number_of_hubs_dict, temperatures, alphas, max_ts_dict):
    """Run the simulated annealing algorithm with various parameters for a certain number of iterations.

    Args:
    iterations: Number of iterations to run the algorithm.
    fm_names: List of flow matrix names.
    number_of_hubs_dict: Dictionary with number of hubs for each flow matrix.
    temperatures: List of initial temperatures for the annealing process.
    alphas: List of cooling rates.
    max_ts_dict: Dictionary with maximum number of iterations for each flow matrix.
    """
    for i in range(iterations):
        print('---------------------------------------------------------------------------------')
        print(f'Iteration: {i}')
        for fm_name in fm_names:
            cm_name = fm_name.replace('fm', 'cm')
            for number_of_hubs in number_of_hubs_dict[fm_name]:
                for max_t in max_ts_dict[fm_name]:
                    for T in temperatures:
                        for alpha in alphas:
                            print(f'fm_name: {fm_name}, cm_name: {cm_name}, number_of_hubs: {number_of_hubs}, max_t: {max_t}, T: {T}, alpha: {alpha}')
                            # Run simulated annealing
                            simulated_annealing(T, alpha, max_t, matrices[fm_name], fm_name, matrices[cm_name], cm_name, number_of_hubs)
                            print('Run completed.')



In [37]:
# Run the algorithm for 10 iterations with different parameters
run_simulated_annealing(10, FM_NAMES, NUMBER_OF_HUBS_DICT, TEMPERATURES, ALPHAS, MAX_TS_DICT)

---------------------------------------------------------------------------------
Iteration: 0
fm_name: CAB_25_fm, cm_name: CAB_25_cm, number_of_hubs: 3, max_t: 2000, T: 1000000000, alpha: 0.96
Timer ended. Total time: 0.35115623474121094
Worse solution accepted: 49 times
Best cost: 8374475034.11352
Run completed.
fm_name: CAB_25_fm, cm_name: CAB_25_cm, number_of_hubs: 3, max_t: 2000, T: 1000000000, alpha: 0.97
Timer ended. Total time: 0.31412720680236816
Worse solution accepted: 64 times
Best cost: 8374475034.11352
Run completed.
fm_name: CAB_25_fm, cm_name: CAB_25_cm, number_of_hubs: 3, max_t: 2000, T: 1000000000, alpha: 0.98
Timer ended. Total time: 0.29454731941223145
Worse solution accepted: 92 times
Best cost: 9524985819.33412
Run completed.
fm_name: CAB_25_fm, cm_name: CAB_25_cm, number_of_hubs: 3, max_t: 2000, T: 1000000000, alpha: 0.99
Timer ended. Total time: 0.3007514476776123
Worse solution accepted: 157 times
Best cost: 8374475034.11352
Run completed.
fm_name: CAB_25_fm, c

Bad pipe message: %s [b'\xa3\x10\xc8\x89\x00\x05\x98\x9e\x0e\xb8!\xea\x85\xd0w\xd2\x10W \xda.\xb0>W\xe2\xe50\x06\xd03<\xeb,\xd0H\x87\x19\x80Z\xea\xee\x1eC\x8fbm\xaa|~\x17\r\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04']
Bad pipe message: %s [b'\x01\x02']
Bad pipe message: %s [b"\xfc\xac\xf8f\xa0[\xd0\xb8\xc0\x00\x9f\xd1s/\xf7\x00TA\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17

Timer ended. Total time: 9.73717999458313
Worse solution accepted: 71 times
Best cost: 136390534183.13239
Run completed.
fm_name: RGP_100_fm, cm_name: RGP_100_cm, number_of_hubs: 10, max_t: 6000, T: 10000000000, alpha: 0.98
Timer ended. Total time: 10.00926661491394
Worse solution accepted: 123 times
Best cost: 134460419105.16681
Run completed.
fm_name: RGP_100_fm, cm_name: RGP_100_cm, number_of_hubs: 10, max_t: 6000, T: 10000000000, alpha: 0.99
Timer ended. Total time: 9.817214012145996
Worse solution accepted: 262 times
Best cost: 133561556095.09578
Run completed.
fm_name: RGP_100_fm, cm_name: RGP_100_cm, number_of_hubs: 10, max_t: 6000, T: 100000000000, alpha: 0.96
Timer ended. Total time: 9.0180184841156
Worse solution accepted: 89 times
Best cost: 133396208039.16748
Run completed.
fm_name: RGP_100_fm, cm_name: RGP_100_cm, number_of_hubs: 10, max_t: 6000, T: 100000000000, alpha: 0.97
Timer ended. Total time: 8.8959059715271
Worse solution accepted: 116 times
Best cost: 133215263206

In [38]:
# Final run with specific parameters
FINAL_TEMPERATURES = [10_000_000_000]
FINAL_ALPHAS = [0.97]
run_simulated_annealing(10, FM_NAMES, NUMBER_OF_HUBS_DICT, FINAL_TEMPERATURES, FINAL_ALPHAS, MAX_TS_DICT)

---------------------------------------------------------------------------------
Iteration: 0
fm_name: CAB_25_fm, cm_name: CAB_25_cm, number_of_hubs: 3, max_t: 2000, T: 10000000000, alpha: 0.97
Timer ended. Total time: 0.2990090847015381
Worse solution accepted: 95 times
Best cost: 8374475034.11352
Run completed.
fm_name: CAB_25_fm, cm_name: CAB_25_cm, number_of_hubs: 5, max_t: 2000, T: 10000000000, alpha: 0.97
Timer ended. Total time: 0.6638531684875488
Worse solution accepted: 90 times
Best cost: 8017861264.586398
Run completed.
fm_name: TR_55_fm, cm_name: TR_55_cm, number_of_hubs: 3, max_t: 2500, T: 10000000000, alpha: 0.97
Timer ended. Total time: 0.3661768436431885
Worse solution accepted: 95 times
Best cost: 28147615297.0
Run completed.
fm_name: TR_55_fm, cm_name: TR_55_cm, number_of_hubs: 5, max_t: 2500, T: 10000000000, alpha: 0.97
Timer ended. Total time: 0.8787834644317627
Worse solution accepted: 82 times
Best cost: 25674801251.999996
Run completed.
fm_name: TR_81_fm, cm_nam